In [ ]:
%%capture 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import tifffile as tiff
import cv2
import os
import gc
from tqdm.notebook import tqdm
import rasterio
from rasterio.windows import Window

from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings("ignore")

import torch
import pytorch_lightning as pl
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

import albumentations as A
from albumentations.pytorch import ToTensorV2

import sys
sys.path.append("../input/timm-pytorch-image-models/pytorch-image-models-master/")
import timm

import cv2
import os
import random
import math
import copy

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
sz = 256   #the size of tiles
reduce = 4 #reduce the original images by 4 times
DATA = '../input/hubmap-kidney-segmentation/test/'
fold_0 = '../input/fold0trained/fold_0_dice.pth'
fold_1 = '../input/fold1trained/fold_1_dice.pth'
fold_2 = '../input/fold2trained/fold_2_dice.pth'
fold_3 = '../input/fold3trained/fold_3_dice.pth'

MODELS = [fold_0, fold_1, fold_2, fold_3]
df_sample = pd.read_csv('../input/hubmap-kidney-segmentation/sample_submission.csv')
bs = 16
test_transforms = A.Compose([
    A.Normalize(),
    ToTensorV2()
])

# Data

In [ ]:
#functions to convert encoding to mask and mask to encoding
def enc2mask(encs, shape):
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for m,enc in enumerate(encs):
        if isinstance(enc,np.float) and np.isnan(enc): continue
        s = enc.split()
        for i in range(len(s)//2):
            start = int(s[2*i]) - 1
            length = int(s[2*i+1])
            img[start:start+length] = 1 + m
    return img.reshape(shape).T

def mask2enc(mask, n=1):
    pixels = mask.T.flatten()
    encs = []
    for i in range(1,n+1):
        p = (pixels == i).astype(np.int8)
        if p.sum() == 0: encs.append(np.nan)
        else:
            p = np.concatenate([[0], p, [0]])
            runs = np.where(p[1:] != p[:-1])[0] + 1
            runs[1::2] -= runs[::2]
            encs.append(' '.join(str(x) for x in runs))
    return encs

#https://www.kaggle.com/bguberfain/memory-aware-rle-encoding
#with transposed mask
def rle_encode_less_memory(img):
    #the image should be transposed
    pixels = img.T.flatten()
    
    # This simplified method requires first and last pixel to be zero
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    
    return ' '.join(str(x) for x in runs)

In [ ]:
# https://www.kaggle.com/iafoss/256x256-images

s_th = 40  #saturation blancking threshold
p_th = 1000*(sz//256)**2 #threshold for the minimum number of pixels
identity = rasterio.Affine(1, 0, 0, 0, 1, 0)

class HuBMAPDataset(Dataset):
    def __init__(self, idx, sz=sz, reduce=reduce):
        self.data = rasterio.open(os.path.join(DATA,idx+'.tiff'),
                                 num_threads='all_cpus')
        # some images have issues with their format 
        # and must be saved correctly before reading with rasterio
        if self.data.count != 3:
            subdatasets = self.data.subdatasets
            self.layers = []
            if len(subdatasets) > 0:
                for i, subdataset in enumerate(subdatasets, 0):
                    self.layers.append(rasterio.open(subdataset))
        self.shape = self.data.shape
        self.reduce = reduce
        self.sz = reduce*sz
        self.pad0 = (self.sz - self.shape[0]%self.sz)%self.sz
        self.pad1 = (self.sz - self.shape[1]%self.sz)%self.sz
        self.n0max = (self.shape[0] + self.pad0)//self.sz
        self.n1max = (self.shape[1] + self.pad1)//self.sz
        
    def __len__(self):
        return self.n0max*self.n1max
    
    def __getitem__(self, idx):
        # the code below may be a little bit difficult to understand,
        # but the thing it does is mapping the original image to
        # tiles created with adding padding, as done in
        # https://www.kaggle.com/iafoss/256x256-images ,
        # and then the tiles are loaded with rasterio
        # n0,n1 - are the x and y index of the tile (idx = n0*self.n1max + n1)
        n0,n1 = idx//self.n1max, idx%self.n1max
        # x0,y0 - are the coordinates of the lower left corner of the tile in the image
        # negative numbers correspond to padding (which must not be loaded)
        x0,y0 = -self.pad0//2 + n0*self.sz, -self.pad1//2 + n1*self.sz
        # make sure that the region to read is within the image
        p00,p01 = max(0,x0), min(x0+self.sz,self.shape[0])
        p10,p11 = max(0,y0), min(y0+self.sz,self.shape[1])
        img = np.zeros((self.sz,self.sz,3),np.uint8)
        # mapping the loade region to the tile
        if self.data.count == 3:
            img[(p00-x0):(p01-x0),(p10-y0):(p11-y0)] = np.moveaxis(self.data.read([1,2,3],
                window=Window.from_slices((p00,p01),(p10,p11))), 0, -1)
        else:
            for i,layer in enumerate(self.layers):
                img[(p00-x0):(p01-x0),(p10-y0):(p11-y0),i] =\
                  layer.read(1,window=Window.from_slices((p00,p01),(p10,p11)))
        
        if self.reduce != 1:
            img = cv2.resize(img,(self.sz//reduce,self.sz//reduce),
                             interpolation = cv2.INTER_AREA)
        
        #check for empty imges
        hsv = cv2.cvtColor(copy.deepcopy(img), cv2.COLOR_BGR2HSV)
        h,s,v = cv2.split(hsv)
    
        # Load in Image Normally
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        img = test_transforms(image = img)['image']
        if (s>s_th).sum() <= p_th or img.sum() <= p_th:
            #images with -1 will be skipped
            del hsv
            return img, -1
        return img, idx
        

# Model

Conv Blocks

In [ ]:
class ConvBlock(pl.LightningModule):
    def __init__(self, in_features, out_features, kernel_size, padding, groups, stride, act = 'relu'):
        super().__init__()
        self.conv = nn.Conv2d(in_features, out_features, kernel_size = kernel_size, padding = padding, groups = groups, stride = stride, bias = False)
        self.bn = nn.BatchNorm2d(out_features)
        if act == 'relu':
            self.act1 = nn.ReLU(inplace = True)
        else:
            self.act1 = nn.SiLU(inplace = True)
    def forward(self, x):
        return self.bn(self.act1(self.conv(x)))
class SqueezeExcite(pl.LightningModule):
    def __init__(self, in_features, inner_features, dev, act = "relu"):
        super().__init__()
        self.in_features = in_features
        self.inner_features = inner_features
        self.dev = dev
        
        self.Squeeze = nn.Linear(self.in_features, self.inner_features)
        if act == 'relu':
            self.act1 = nn.ReLU(inplace = True)
        else:
            self.act1 = nn.SiLU(inplace = True)
        self.Excite = nn.Linear(self.inner_features, self.in_features)
        
        self.gamma = nn.Parameter(torch.zeros((1), device = self.dev))
    def forward(self, x):
        mean = torch.mean(x, dim = -1)
        mean = torch.mean(mean, dim = -1)
        
        squeeze = self.act1(self.Squeeze(mean))
        excite = torch.sigmoid(self.Excite(squeeze)).unsqueeze(-1).unsqueeze(-1)
        return excite * x * self.gamma + (1 - self.gamma) * x 
class CBAMChannel(pl.LightningModule):
    def __init__(self, in_features, inner_features, dev, act = 'relu'):
        super().__init__()
        self.in_features = in_features
        self.inner_features = inner_features
        self.dev = dev 
        
        self.Squeeze = nn.Linear(self.in_features, self.inner_features) 
        if act == 'relu':
            self.act1 = nn.ReLU(inplace = True)
        else:
            self.act1 = nn.SiLU(inplace = True)
        self.Excite = nn.Linear(self.inner_features, self.in_features)
        self.gamma = nn.Parameter(torch.zeros((1), device = self.dev))
    def forward(self, x):
        mean = torch.mean(x, dim = -1)
        mean = torch.mean(mean, dim = -1)
        
        max_pool, _ = torch.max(x, dim = -1) 
        max_pool, _ = torch.max(max_pool, dim = -1)
        
        squeeze_mean = self.act1(self.Squeeze(mean))
        excite_mean = self.Excite(squeeze_mean)
        
        squeeze_max = self.act1(self.Squeeze(max_pool))
        excite_max = self.Excite(squeeze_max)
        
        excite = torch.sigmoid((excite_mean + excite_max) / 2).unsqueeze(-1).unsqueeze(-1)
        return excite * x * self.gamma + (1 - self.gamma) * x
        
class Attention(pl.LightningModule):
    def __init__(self, in_features, inner_features, dev, attention_type = 'se', act = 'relu'):
        super().__init__()
        self.attention_type = attention_type
        assert self.attention_type in ['se', 'cbam', 'none']
        if self.attention_type == 'se':
            self.layer = SqueezeExcite(in_features, inner_features, dev, act = act)
        elif self.attention_type == 'cbam':
            self.layer = CBAMChannel(in_features, inner_features, dev, act = act)
        else:
            self.layer= nn.Identity()
    def forward(self, x):
        return self.layer(x)

# Self Attention Blocks
class ConvPlusBatchNorm(pl.LightningModule):
    '''
    Conv2d + BN, no activation.
    '''
    def __init__(self, in_features, out_features, kernel_size, padding, groups, stride):
        super().__init__()
        self.conv = nn.Conv2d(in_features, out_features, kernel_size = kernel_size, padding = padding, groups = groups, stride = stride)
        self.bn1 = nn.BatchNorm2d(out_features)
    def forward(self, x):
        return self.bn1(self.conv(x))
class SelfAttention(pl.LightningModule):
    # Non Local Block.
    def __init__(self, in_features, inner_features, num_heads):
        super().__init__()
        self.in_features = in_features
        self.inner_features = inner_features
        self.num_heads = num_heads
        self.K = ConvPlusBatchNorm(self.in_features, self.inner_features * self.num_heads, 3, 1, 1, 1)
        self.V = ConvPlusBatchNorm(self.in_features, self.inner_features * self.num_heads, 3, 1, 1, 1)
        self.Q = ConvPlusBatchNorm(self.in_features, self.inner_features * self.num_heads, 3, 1, 1, 1)
        self.Linear = ConvPlusBatchNorm(self.inner_features * self.num_heads, self.in_features, 3, 1, 1, 1)
        
        self.gamma = nn.Parameter(torch.zeros((1), device = self.device))
    def forward(self, x):
        B, C, H, W = x.shape
        Keys = self.K(x)
        Values = self.V(x)
        Queries = self.Q(x)
        
        Keys = Keys.reshape(B, self.num_heads, self.inner_features, H, W)
        Values = Values.reshape(B, self.num_heads, self.inner_features, H, W)
        Queries = Queries.reshape(B, self.num_heads, self.inner_features, H, W) 
        
        Keys = Keys.reshape(B * self.num_heads, self.inner_features, H * W)
        Values = Values.view(B * self.num_heads, self.inner_features, H * W)
        Queries = Queries.view(B * self.num_heads, self.inner_features, H * W)
        
        att_mat = F.softmax(torch.bmm(Keys.transpose(1, 2), Queries) / math.sqrt(self.inner_features))
        att_vals = torch.bmm(att_mat, Values.transpose(1, 2))
        
        scores = att_vals.view(B, self.num_heads, self.inner_features, H, W)
        scores = scores.view(B, self.num_heads * self.inner_features, H, W)
        output = self.Linear(scores) 
        return output * self.gamma + (1 - self.gamma) * x
class CBAMSqueezeAttend(pl.LightningModule):
    def __init__(self, in_features, inner_features, out_features, out_size, squeeze_factor = 4, act = 'relu'):
        super().__init__()
        self.in_features = in_features
        self.inner_features = inner_features
        self.out_features = out_features
        self.out_size = out_size
        self.squeeze_factor = squeeze_factor
        self.act = act 
        
        self.proj = ConvBlock(self.in_features, self.out_features, 3, 1, 1, 1, act = self.act)
        self.max_pool = nn.MaxPool2d(kernel_size = 5, padding = 2, stride = self.squeeze_factor)
        self.avg_pool = nn.AvgPool2d(kernel_size = 5, padding = 2, stride = self.squeeze_factor)
        
        self.Squeeze = ConvBlock(self.out_features, self.inner_features, 3, 1, 1, 1, act = self.act)
        self.Excite = ConvPlusBatchNorm(self.inner_features, self.out_features, 3, 1, 1, 1)
        
        self.gamma = nn.Parameter(torch.zeros((1), device = self.device))
    def forward(self, x):
        B, C, H, W = x.shape
        x = self.proj(x)
        
        max_pool = self.max_pool(x)
        avg_pool = self.avg_pool(x)
        
        squeeze_max = self.Squeeze(max_pool)
        squeeze_avg = self.Squeeze(avg_pool) 
        
        excite_max = self.Excite(squeeze_max)
        excite_avg = self.Excite(squeeze_avg)
        
        excite = torch.sigmoid((excite_max + excite_avg) / 2)
        excited = avg_pool * self.gamma * excite + (1 - self.gamma) * avg_pool
    
        # Interpolate Upward
        excited = F.interpolate(excited, size = (self.out_size, self.out_size), mode = 'nearest')
        return excited
class SESqueezeAttend(pl.LightningModule):
    def __init__(self, in_features, inner_features, out_features, out_size, squeeze_factor = 4, act = 'relu'):
        super().__init__()
        self.in_features = in_features
        self.inner_features = inner_features
        self.out_size = out_size
        self.squeeze_factor = squeeze_factor 
        self.act = act
        self.avg_pool = nn.AvgPool2d(kernel_size = 5, padding = 2, stride = squeeze_factor)
        
        self.proj = ConvBlock(self.in_features, self.out_features, 3, 1, 1, 1, act = self.act)
        self.Squeeze = ConvBlock(self.in_features, self.inner_features, 3, 1, 1, 1, act = self.act)
        self.Excite = ConvPlusBatchNorm(self.inner_features, self.in_features, 3, 1, 1, 1)
        
        self.gamma = nn.Parameter(torch.zeros((1), device = self.device))
    def forward(self, x):
        '''
        x: Tensor(B, C, H, W)
        '''
        B, C, H, W = x.shape
        x = self.proj(x)
        pooled = self.avg_pool(x)
        
        squeeze = self.Squeeze(pooled)
        excite = torch.sigmoid(self.Excite(squeeze))
        
        excited = self.gamma * pooled * excite + (1 - self.gamma) * pooled 
        
        # Interpolate Back Up.
        excited = F.interpolate(excited, size = (self.out_size, self.out_size), mode = 'nearest')
        return excited
class SqueezeAttend(pl.LightningModule):
    def __init__(self, in_features, inner_features, out_features, out_size, squeeze_factor = 4, act = 'relu', attention_type = 'se'):
        super().__init__()
        
        self.attention_type = attention_type
        assert self.attention_type in ['se', 'cbam', 'none']
        if self.attention_type == 'se':
            self.layer = SESqueezeAttend(in_features, inner_features, out_features, out_size, squeeze_factor = squeeze_factor, act = act)
        elif self.attention_type =='cbam':
            self.layer = CBAMSqueezeAttend(in_features, inner_features, out_features, out_size, squeeze_factor = squeeze_factor, act = act)
        else:
            self.layer = nn.Identity()
    def forward(self,x):
        return self.layer(x)

class BottleNeckBlock(pl.LightningModule):
    def __init__(self, in_features, inner_features, dev, attention_type = 'se', stochastic_depth = 0, act = 'relu'):
        super().__init__()
        self.stochastic_depth = stochastic_depth
        self.in_features = in_features
        self.inner_features = inner_features
        self.dev = dev
        self.attention_type = attention_type
        
        self.Squeeze = ConvBlock(self.in_features, self.inner_features, 1, 0, 1, 1, act = act) 
        self.Process = ConvBlock(self.inner_features, self.inner_features, 3, 1, 1, 1, act = act)
        self.Expand = ConvBlock(self.inner_features, self.in_features, 1, 0, 1, 1, act = act)
        self.SE = Attention(self.in_features, self.in_features // 4, self.dev, attention_type = self.attention_type)

        self.gamma = nn.Parameter(torch.zeros((1), device = self.dev))
    def forward(self, x):
        if self.training and random.random() < self.stochastic_depth:
            return x
        squeeze = self.Squeeze(x)
        process = self.Process(squeeze)
        expand = self.Expand(process)
        SE = self.SE(expand)
        return SE * self.gamma + (1 - self.gamma) * x
        
class InverseBottleNeckBlock(pl.LightningModule):
    def __init__(self, in_features, inner_features, dev, attention_type = 'se', stochastic_depth = 0, act = 'relu'):
        super().__init__()
        self.stochastic_depth = stochastic_depth
        self.in_features = in_features
        self.inner_features = inner_features
        self.dev = dev
        self.attention_type = attention_type
        
        self.Expand = ConvBlock(self.in_features, self.inner_features, 1, 0, 1, 1, act = act) 
        self.DW = ConvBlock(self.inner_features, self.inner_features, 3, 1, self.inner_features, 1, act = act)
        self.SE = Attention(self.inner_features, self.inner_features//4, self.dev, attention_type = self.attention_type, act = act)
        self.Squeeze = ConvBlock(self.inner_features, self.in_features, 1, 0, 1, 1, act = act)
        
        self.gamma = nn.Parameter(torch.zeros((1), device = self.device))
    def forward(self, x):
        if self.training and random.random() < self.stochastic_depth:
            return x
        expand = self.Expand(x)
        dw = self.DW(expand)
        se = self.SE(dw)
        squeeze = self.Squeeze(se)
        return squeeze * self.gamma + (1 - self.gamma) * x

class DownSamplerBottleNeck(pl.LightningModule):
    def __init__(self, in_features, inner_features, out_features, stride, dev, attention_type = 'se', act = 'relu'):
        super().__init__()
        self.in_features = in_features
        self.inner_features = inner_features
        self.out_features = out_features
        self.stride = stride
        self.dev = dev
        self.attention_type = attention_type
        
        self.pool = nn.AvgPool2d(kernel_size = 3, padding = 1, stride =stride)
        self.pool_conv = ConvBlock(self.in_features, self.out_features, 1, 0, 1, 1, act = act)
        self.Squeeze = ConvBlock(self.in_features, self.inner_features, 1, 0, 1, 1, act = act) 
        self.Process = ConvBlock(self.inner_features, self.inner_features, 3, 1, 1, 1, act = act)
        self.Expand = ConvBlock(self.inner_features, self.out_features, 1, 0, 1, self.stride, act = act)
        self.SE = Attention(self.out_features, self.out_features // 4, self.dev, attention_type = self.attention_type, act = act)
        
        self.gamma = nn.Parameter(torch.zeros((1), device = self.dev))
    def forward(self, x):
        pool = self.pool_conv(self.pool(x))
        conv_features = self.SE(self.Expand(self.Process(self.Squeeze(x))))
        return pool * self.gamma + pool * (1 - self.gamma)
class DownSamplerInverse(pl.LightningModule):
    def __init__(self, in_features, inner_features, out_features, stride, dev, attention_type = 'se', act = 'relu'):
        super().__init__()
        self.in_features = in_features
        self.inner_features = inner_features
        self.out_features = out_features
        self.stride = stride
        self.dev = dev
        self.attention_type = attention_type
    
        self.pool = nn.AvgPool2d(kernel_size = 3, padding = 1, stride = self.stride)
        self.pool_conv = ConvBlock(self.in_features, self.out_features, 1, 0, 1, 1, act = act)
        
        self.squeeze = ConvBlock(self.in_features, self.inner_features,1, 0, 1, 1, act = act)
        self.process = ConvBlock(self.inner_features, self.inner_features, 3, 1, self.inner_features, 1, act = act)
        self.SE = Attention(self.inner_features, self.inner_features // 4, self.dev, act = act)
        self.expand = ConvBlock(self.inner_features, self.out_features, 1, 0, 1, self.stride, act = act)
        
        self.gamma = nn.Parameter(torch.zeros((1), device = self.device))
    def forward(self, x):
        pool = self.pool_conv(self.pool(x))
        conv = self.expand(self.SE(self.process(self.squeeze(x))))
        return conv * self.gamma + (1 - self.gamma) * pool

class AstrousConvolution(pl.LightningModule):
    '''
    Astrous(More Properly - à trous(at holes in french)) Convolution
    '''
    def __init__(self, in_features, out_features, kernel_size, padding, groups, stride, dilation, act = 'relu'):
        super().__init__()
        self.astrous = nn.Conv2d(in_features, out_features, kernel_size = kernel_size, padding = padding, groups = groups, stride = stride, dilation = dilation, bias = False)
        self.bn = nn.BatchNorm2d(out_features)
        if act == 'relu':
            self.act1 = nn.ReLU(inplace = True)
        else:
            self.act1 = nn.SiLU(inplace = True)
    def forward(self, x):
        return self.bn(self.act1(self.astrous(x)))
class ASPP(pl.LightningModule):
    '''
    à trous spatial pooling pyramid block. No further Processing, this should be added later.
    
    5 Part:
    - Normal Conv
    - à trous: 3 dilation
    - à trous: 5 dilation
    - à trous: 7 dilation
    '''
    def __init__(self, in_features, inner_features, out_features, act = 'relu'):
        super().__init__()
        self.in_features = in_features
        self.inner_features = inner_features
        self.out_features = out_features
        self.conv1 = ConvBlock(self.in_features, self.inner_features, 3, 1, 1, 2, act = act)
        self.conv2 = AstrousConvolution(self.in_features, self.inner_features, 3, 1, 1, 1, 3, act = act)
        self.conv3 = AstrousConvolution(self.in_features, self.inner_features, 3, 3, 1, 1, 5, act = act)
        self.conv4 = AstrousConvolution(self.in_features, self.inner_features, 3, 5, 1, 1, 7, act = act) 
        
        self.proj = ConvBlock(4 * self.inner_features, self.out_features, 1, 0, 1, 1, act = act)
        
    def forward(self, x):
        conv1 = self.conv1(x)
        conv2 = self.conv2(x)
        conv3 = self.conv3(x)
        conv4 = self.conv4(x)
        # concat
        concat = torch.cat([conv1, conv2, conv3, conv4], dim = 1)
        proj = self.proj(concat)
        return proj

Encoders

In [ ]:
# ResNet Based Complex Encoder(+ SE + dropout2d)
class ResNetEncoderAlpha(pl.LightningModule):
    '''
    ResNet34d encoder + SE and Dropout
    
    I would scale the model larger(ex. ResNet50), but larger models have the wrong dimensions.
    '''
    def freeze(self, layers):
        for layer in layers:
            for parameter in layer.parameters():
                parameter.requires_grad = False
    def unfreeze(self, layers):
        for layer in layers:
            for parameter in layer.parameters():
                parameter.requires_grad = True 
    def increase_drop(self):
        self.drop_prob += self.increase_dropout
    def __init__(self, increase_drop, attention_type):
        super().__init__()
        # HYPER PARAMETERS
        self.increase_dropout = increase_drop
        self.attention_type = attention_type
        self.drop_prob = 0
        self.model_name = 'resnet34d'
        # END OF HYPER PARAMETERS
        self.model = timm.create_model(self.model_name, pretrained = False) 
        # Extract Layers
        self.conv1 = self.model.conv1 # (B, 64, 128, 128)
        self.bn1 = self.model.bn1
        self.act1 = self.model.act1
        self.maxpool = self.model.maxpool
        
        self.layer1 = self.model.layer1 # (b, 64, 64, 64)
        self.layer2 = self.model.layer2 # (b, 128, 32, 32)
        self.layer3 = self.model.layer3 # (b, 256, 16, 16)
        self.layer4 = self.model.layer4 # (b, 512, 8, 8)
        
        self.Dropout0 = nn.Dropout2d(self.drop_prob)
        self.Attention0 = Attention(64, 16, self.device, attention_type = self.attention_type)
        self.increase_drop()
        
        self.Dropout1 = nn.Dropout2d(self.drop_prob)
        self.increase_drop()
        self.Attention1 = Attention(64, 16, self.device, attention_type = self.attention_type)
        self.Dropout2 = nn.Dropout2d(self.drop_prob)
        self.Attention2 = Attention(128, 32, self.device, attention_type = self.attention_type)
        self.increase_drop()
        self.Dropout3 = nn.Dropout2d(self.drop_prob)
        self.Attention3 = Attention(256, 64, self.device, attention_type = self.attention_type)
        self.increase_drop()
        self.Dropout4 = nn.Dropout2d(self.drop_prob)
        self.Attention4 = Attention(512, 128, self.device, attention_type = self.attention_type)
        
        del self.model
    def forward(self, x):
        features0 = self.bn1(self.act1(self.conv1(x)))
        features0 = self.Dropout0(features0)
        features0 = self.Attention0(features0)
        
        layer1 = self.layer1(self.maxpool(features0))
        layer1 = self.Dropout1(layer1)
        layer1 = self.Attention1(layer1)
        
        layer2 = self.layer2(layer1)
        layer2 = self.Dropout2(layer2)
        layer2 = self.Attention2(layer2)
        
        layer3 = self.layer3(layer2)
        layer3 = self.Dropout3(layer3)
        layer3 = self.Attention3(layer3)
        
        layer4 = self.layer4(layer3)
        layer4 = self.Dropout4(layer4)
        layer4 = self.Attention4(layer4)
        
        return x, features0, layer1, layer2, layer3, layer4
class EffNetEncoderAlpha(pl.LightningModule):
    '''
    EfficientNet-b4 based Encoder(SE + Dropout)
    '''
    def freeze(self, layers):
        for layer in layers:
            for parameter in layer.parameters():
                parameter.requires_grad = False
    def unfreeze(self, layers):
        for layer in layers:
            for parameter in layer.parameters():
                parameter.requires_grad = True 
    def increase_drop(self):
        self.drop_prob += self.increase_drop
    def __init__(self, increase_drop, attention_type, act = 'relu'):
        super().__init__()
        self.drop_prob = 0
        self.act = act 
        self.increase_drop = increase_drop
    
        self.attention_type = attention_type
        
        self.model_name = 'tf_efficientnet_b4_ns'
        self.model = timm.create_model(self.model_name, pretrained = True)
        
        self.conv1 = self.model.conv_head
        self.bn1 = self.model.bn1
        self.act1 = self.model.act1
    
        self.block0 = self.model.blocks[0] # 24
        self.block1 = self.model.blocks[1] # 32
        self.block2 = self.model.blocks[2] # 56
        self.block3 = self.model.blocks[3] # 112
        self.block4 = self.model.blocks[4] # 160 
        self.block5 = self.model.blocks[5] # 272
        self.block6 = self.model.blocks[6] # 448
        
        # Custom Layer
        self.Dropout0 = nn.Dropout2d(self.drop_prob)
        self.Attention0 = Attention(24, 6, self.device, attention_type= self.attention_type, act = self.act)
        self.increase_dropout()
        
        self.Dropout1 = nn.Dropout2d(self.drop_prob)
        self.Attention1 = Attention(32, 8, self.device, attention_type = self.attention_type, act = self.act)
        self.increase_dropout()
        
        self.Dropout2 = nn.Dropout2d(self.drop_prob)
        self.Attention2 = Attention(56, 16, self.device, attention_type = self.attention_type, act = self.act)
        self.increase_dropout()
    
        self.Dropout3 = nn.Dropout2d(self.drop_prob)
        self.Attention3 = Attention(160, 48, self.device, attention_type = self.attention_type, act = self.act)
        self.increase_dropout()
        
        self.Dropout4 = nn.Dropout2d(self.drop_prob)
        self.Attention4 = Attention(448, 128, self.device, attention_type = self.attention_type, act = self.act)
        
        # Proj Blocks(To Match ResBlocks)
        self.proj0 = ConvBlock(24, 64, 3, 1, 1, 1)
        self.proj1 = ConvBlock(32, 64, 3, 1, 1, 1)
        self.proj2 = ConvBlock(56, 128, 3, 1, 1, 1)
        self.proj3 = ConvBlock(160, 256, 3, 1, 1, 1)
        self.proj4 = ConvBlock(448, 512, 3, 1, 1, 1)
    def forward(self, x):
        '''
        l0: (b, 3, 256, 256)
        l1: (B, 64, 128, 128)
        l2: (B, 64, 64, 64)
        l3: (B, 128, 32, 32)
        l4: (B, 256, 16, 16)
        l5: (B, 512, 8, 8)
        '''
        features0 = self.bn1(self.act1(self.conv1(x))) # (B, 48, 128, 128)
        block0 = self.block0(features0) # (B, 24, 128, 128)
        block0 = self.Dropout0(block0)
        block0 = self.Attention0(block0)
        
        block1 = self.block1(block0) # (B, 32, 64, 64)
        block1 = self.Dropout1(block1)
        block1 = self.Attention1(block1)
        
        block2 = self.block2(block1) # (B, 56, 32, 32)
        block2 = self.Dropout2(block2)
        block2 = self.Attention2(block2)
        
        block3 = self.block3(block2) # (B, 112, 16, 16)
        block4 = self.block4(block3) # (B, 160, 16, 16)
        block4 = self.Dropout3(block4)
        block4 = self.Attention3(block4)
        
        block5 = self.block5(block4) # (B, 272, 8, 8)
        block6 = self.block6(block5) # (B, 448, 8, 8)
        block6 = self.Dropout4(block6)
        block6 = self.Attention4(block6)
        
        # Project Block
        l1 = self.proj0(block0)
        l2 = self.proj1(block1)
        l3 = self.proj2(block2)
        l4 = self.proj3(block4)
        l5 = self.proj4(block6)
        return x, l1, l2, l3, l4, l5   
        

class EncoderQTPi(pl.LightningModule):
    def freeze(self, layers):
        for layer in layers:
            for parameter in layer.parameters():
                parameter.requires_grad = False
    def unfreeze(self, layers):
        for layer in layers:
            for parameter in layer.parameters():
                parameter.requires_grad = True
    def increase_dropout(self):
        self.drop_prob += self.increase_drop
    def increase_stochasticity(self):
        '''
        Increases the Rate of Stochastic Depth Dropout(Deeper should drop more.)
        '''
        self.stochastic_depth += self.increase_stoc
    def __init__(self, increase_drop, increase_stoc, attention_type, use_ASPP = False, encoder_type = 'resnet', act = 'relu'):
        # Suggested Increase_drop = 0.05, increase_stoc = 0.1
        super().__init__()
        self.act = act
        self.encoder_type = encoder_type
        assert self.encoder_type in ['resnet', 'effnet']
        self.increase_drop = increase_drop
        self.drop_prob = 5 * self.increase_drop
        
        self.stochastic_depth = 0
        self.increase_stoc = increase_stoc
        
        self.attention_type = attention_type
        
        self.backbone = ResNetEncoderAlpha(self.increase_drop, self.attention_type) if self.encoder_type == 'resnet' else EffNetEncoderAlpha(self.increase_drop, self.attention_type, act = self.act)
        
        self.use_ASPP = use_ASPP
        
        def add_block_stoc(x):
            self.increase_stochasticity()
            return x
        def add_block(x):
            # Adds a Block and Increases the Stochasticity of the model
            self.increase_dropout()
            self.increase_stochasticity()
            return x
        if self.use_ASPP:
            self.ASPP = nn.Sequential(*[
                ASPP(512, 256, 1024, act = self.act)
            ] + [
                add_block_stoc(BottleNeckBlock(1024, 256, self.device, attention_type = self.attention_type, stochastic_depth = self.stochastic_depth, act = self.act)) for i in range(3)
            ])
            
        else:
            self.ASPP = nn.Sequential(*[
                DownSamplerBottleNeck(512, 256, 1024, 2, self.device, attention_type = self.attention_type, act = self.act),
            ] + [
                add_block_stoc(BottleNeckBlock(1024, 256, self.device, attention_type = self.attention_type, stochastic_depth = self.stochastic_depth, act = self.act)) for i in range(3)
            ])
        
        self.layer7 = nn.Sequential(*[
            DownSamplerBottleNeck(1024, 512, 2048, 2, self.device, attention_type = self.attention_type, act = self.act)
        ] + [
            add_block_stoc(BottleNeckBlock(2048, 512, self.device, attention_type= self.attention_type, stochastic_depth = self.stochastic_depth, act = self.act)) for i in range(2)
        ])
        
        self.Dropout6 = nn.Dropout2d(self.drop_prob)
        self.increase_dropout()
        self.Attention6 = Attention(1024, 256, self.device, attention_type = self.attention_type, act = self.act)
        
        self.Dropout7 = nn.Dropout2d(self.drop_prob)
        self.increase_dropout()
        self.Attention7 = Attention(2048, 512, self.device, attention_type = self.attention_type, act = self.act)
        
    def forward(self, x):
        l0, l1, l2, l3, l4, l5 = self.backbone(x) 
        # L5: (B, 512, 8, 8) 
        l6 = self.ASPP(l5)
        l6 = self.Dropout6(l6)
        l6 = self.Attention6(l6)
        
        l7 = self.layer7(l6)
        l7 = self.Dropout7(l7)
        l7 = self.Attention7(l7)
        return l0, l1, l2, l3, l4, l5, l6, l7

Base Line Model

In [ ]:
class EncoderBaseLine(pl.LightningModule):
    '''
    ResNet34 Pretrained Model
    '''
    def freeze(self, layers):
        for layer in layers:
            for parameter in layer.parameters():
                parameter.requires_grad = False
    def unfreeze(self, layers):
        for layer in layers:
            for parameter in layer.parameters():
                parameter.requires_grad = True
    def __init__(self):
        super().__init__()
        self.model_name = 'resnet34d'
        self.model = timm.create_model(self.model_name, pretrained = True)
        # Extract Layers
        self.conv1 = self.model.conv1
        self.bn1 = self.model.bn1
        self.act1 = self.model.act1
        self.pool = self.model.maxpool
        
        self.layer1 = self.model.layer1
        self.layer2 = self.model.layer2
        self.layer3 = self.model.layer3
        self.layer4 = self.model.layer4
        # Freeze Initial Layers
        #self.freeze([self.conv1, self.bn1, self.layer1])
    def forward(self, x):
        features0 = self.bn1(self.act1(self.conv1(x))) # (B, 64, 128, 128)
        
        layer1 = self.layer1(self.pool(features0)) # (B, 64, 64, 64)
        layer2 = self.layer2(layer1) # (B, 128, 32, 32)
        layer3 = self.layer3(layer2) # (B, 256, 16, 16)
        layer4 = self.layer4(layer3) # (B, 512, 8, 8)
        return x, features0, layer1, layer2, layer3, layer4

class BaseLineUNetBlock(pl.LightningModule):
    '''
    UNet Block, upsamples using interpolation(Transposed Convolutions are very unstable and annoying to deal with.)
    '''
    def __init__(self, left_features, down_features, out_features, act = 'relu'):
        super().__init__()
        self.act = act
        self.left_features = left_features
        self.down_features = down_features
        self.out_features = out_features
        
        self.proj = ConvBlock(self.left_features + self.down_features, self.out_features, 3, 1, 1, 1, act = self.act) 
        
        self.process = ConvBlock(self.out_features, self.out_features, 3, 1, 1, 1, act = self.act)
        
    def forward(self, left_features, down_features):
        B, C, H, W = down_features.shape
        upsampled = F.interpolate(down_features, scale_factor= 2, mode = 'nearest') # Upsample images
        if left_features != None:
            upsampled = torch.cat([left_features, upsampled], dim = 1) # Concatenate
        features = self.proj(upsampled)
        return self.process(features)

class DecoderBaseLine(pl.LightningModule):
    '''
    Decoder with nothing Fancy. For Testing and Sanity Check
    '''
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes
    
        self.left_features = [256, 128, 64, 64, 0]
        self.down_dims = [512, 256, 128, 64, 32, 16]
        self.dec_blocks = nn.ModuleList([
            BaseLineUNetBlock(self.left_features[i], self.down_dims[i], self.down_dims[i + 1]) for i in range(len(self.left_features))
        ])
        self.proj = nn.Conv2d(16, 1, kernel_size = 3, padding =1)
    def forward(self, l0, l1, l2, l3, l4, l5):
        '''
        Encoder Dims:
        [B, 3, 256, 256]
        [B, 64, 128, 128],
        [B, 64, 64, 64],
        [B, 128, 32, 32],
        [B, 256, 16, 16]
        [B, 512, 8, 8]
        '''
        d4 = self.dec_blocks[0](l4, l5)
        d3 = self.dec_blocks[1](l3, d4)
        d2 = self.dec_blocks[2](l2, d3)
        d1 = self.dec_blocks[3](l1, d2)
        d0 = self.dec_blocks[4](None, d1)
        return self.proj(d0)
        

class BaseLineSolution(pl.LightningModule):
    def __init__(self):
        super().__init__()
        # HYPER PARAMETERS-----------------
        self.num_classes = 1
        # END OF HYPER PARAMETERS ---------
        self.encoder = EncoderBaseLine()
        self.decoder = DecoderBaseLine(self.num_classes) 
    def forward(self, x):
        return torch.squeeze(self.decoder(*self.encoder(x)))

Decoder

In [ ]:
class FPN(pl.LightningModule):
    '''
    Feature Pyramid Network, incorporates information at all scales of the network
    '''
    def __init__(self, in_features, out_features, out_size, attention_type = 'se', act = 'relu'):
        super().__init__()
        self.in_features = in_features
        self.num_blocks = len(self.in_features)
        self.out_features = out_features
        self.out_size = out_size
        self.act = act
        self.attention_type = attention_type
        
        self.SABlocks = nn.ModuleList([
            SqueezeAttend(self.in_features[i], self.out_features // 4, self.out_features, self.out_size, act = self.act, attention_type = self.attention_type) for i in range(self.num_blocks)   
        ])
        self.proj = ConvBlock(self.num_blocks * self.out_features, self.out_features, 3, 1, self.out_features, 1)
        
    def forward(self, x):
        assert isinstance(x, list) and len(x) == self.num_blocks
        # Process Each of the Features
        features = []
        for i in range(self.num_blocks):
            features += [self.SABlocks[i](x[i])]
        # Concatenate
        concat = torch.cat(features, dim = 1)
        return self.proj(concat)
        
        

class DecoderBlockQTPi(pl.LightningModule):
    '''
    Uses Pixel Shuffle, Concatenation, and Attention to Upsample Blocks(Mimic ResNet on the Way up)
    '''
    def increase_stochasticity(self):
        self.stochastic_depth += self.increase_stochastic
    def __init__(self, left_features, down_features, out_features, num_blocks, attention_type, drop_prob, use_pixel_shuffle = True, act = 'relu', stochastic_depth = 0, increase_stochastic = 0):
        super().__init__()
        self.left_features = left_features
        self.out_features = out_features
        self.stochastic_depth = stochastic_depth
        self.increase_stochastic = increase_stochastic
        self.act = act
        self.num_blocks = num_blocks
        self.down_features = down_features
        self.use_pixel_shuffle = use_pixel_shuffle
        self.attention_type = attention_type
        self.drop_prob = drop_prob
        assert self.down_features % 4 == 0
        
        if self.use_pixel_shuffle:
            self.pixel_shuffle = nn.PixelShuffle(2)
            self.att1 = Attention(self.down_features // 4, self.down_features // 16, self.device, act = self.act, attention_type = self.attention_type)
            self.concat_dim = self.left_features + self.down_features // 4
        else:
            self.concat_dim = self.down_features + self.left_features
            self.att1= Attention(self.down_features, self.down_features // 4, self.device, act = self.act, attention_type = self.attention_type)
    
        self.proj = ConvBlock(self.concat_dim, self.out_features, 3, 1, 1, 1)
        
        def add_block(x):
            self.increase_stochasticity()
            return x
        self.blocks = nn.Sequential(*[
            add_block(BottleNeckBlock(self.out_features, self.out_features // 4, self.device, attention_type = self.attention_type, act= self.act, stochastic_depth = self.stochastic_depth)) for i in range(self.num_blocks)
        ])
        self.dropout = nn.Dropout2d(self.drop_prob)
        self.att2 = Attention(self.out_features, self.out_features // 4, self.device, act = self.act, attention_type = self.attention_type)
    
        
    def forward(self, left_features, down_features):
        '''
        x: Tensor(B, C, H, W) 
        '''
        if self.use_pixel_shuffle:
            # Pixel Shuffle Upsample Down Features
            upsampled = self.pixel_shuffle(down_features)
        else:
            upsampled = F.interpolate(down_features, scale_factor = 2, mode = 'nearest')
        upsampled = self.att1(upsampled)
        if left_features != None:
            concat = torch.cat([left_features, upsampled], dim = 1)
        else:
            concat = upsampled # Final Layer
        proj = self.proj(concat)
        blocks = self.blocks(proj)
        dropped = self.dropout(blocks)
        return self.att2(dropped)
        
        

class DecoderQTPi(pl.LightningModule):
    def increase_stochasticity(self):
        self.stochastic_depth += self.increase_stochastic
    def increase_dropout(self):
        self.drop_prob += self.increase_drop
    def __init__(self, num_classes, attention_type, act, use_pixel_shuffle = True, drop_prob = 0, increase_drop = 0, stochastic_depth = 0, increase_stochastic = 0):
        super().__init__()
        self.drop_prob = drop_prob
        self.increase_drop = increase_drop
        self.use_pixel_shuffle = use_pixel_shuffle
        self.stochastic_depth = stochastic_depth
        self.increase_stochastic = increase_stochastic
        self.attention_type = attention_type
        self.act = act
        self.num_classes = num_classes
    
        self.left_features = [1024, 512, 256, 128, 64, 64, 0]
        self.down_dims = [2048, 1024, 512, 256, 128, 64, 64, 16]
        self.num_blocks = [5, 4, 4, 3, 3, 2, 2] # slightly Mimics ResNet's Block Structure
        
        def add_block(i):
            block = DecoderBlockQTPi(self.left_features[i], self.down_dims[i], self.down_dims[i + 1], self.num_blocks[i], self.attention_type, self.drop_prob, stochastic_depth = self.stochastic_depth, increase_stochastic = self.increase_stochastic, act = self.act, use_pixel_shuffle = self.use_pixel_shuffle)
            for x in range(self.num_blocks[i]):
                self.increase_stochasticity()
            self.increase_dropout()
            return block
        self.dec_blocks = nn.ModuleList([
           add_block(i) for i in range(len(self.left_features))
        ])
        
        # FPN layers
        self.FPN = FPN(self.down_dims[1:-2], 64, out_size = 128, attention_type = self.attention_type, act = self.act) 
        self.proj_FPN = ConvBlock(128, 64, 3, 1, 64, 1)
        
        self.proj = nn.Conv2d(self.down_dims[-1], self.num_classes, kernel_size = 3, padding =1, bias = False)
    def forward(self, l0, l1, l2, l3, l4, l5, l6, l7):
        '''
        Encoder Dims:
        [B, 3, 256, 256]
        [B, 64, 128, 128],
        [B, 64, 64, 64],
        [B, 128, 32, 32],
        [B, 256, 16, 16]
        [B, 512, 8, 8]
        '''
        d6 = self.dec_blocks[0](l6, l7)
        d5 = self.dec_blocks[1](l5, d6)
        d4 = self.dec_blocks[2](l4, d5)
        d3 = self.dec_blocks[3](l3, d4)
        d2 = self.dec_blocks[4](l2, d3)
        d1 = self.dec_blocks[5](l1, d2)
        
        
        fpn = self.FPN([d6, d5, d4, d3, d2])
        # Concatenate with the d1
        concat = torch.cat([fpn, d1], dim = 1) 
        fpn_proj = self.proj_FPN(concat)
        
        d0 = self.dec_blocks[6](None, fpn_proj)
        return self.proj(d0)
        
        

Full Model and Solver

In [ ]:
class UNetQTPi(pl.LightningModule):
    def __init__(self):
        super().__init__()
        # Params
        self.increase_drop_prob = 0.05
        self.stochastic_depth = 0.1
        self.num_classes = 1
        self.attention_type = 'cbam'
        self.model_type = 'resnet'
        self.act = 'relu'
        self.use_ASPP = True
        
        self.use_pixel_shuffle = False
        self.decoder_stoc = 0
        self.decoder_increase_stoc = 0.0
        self.decoder_drop = 0
        self.decoder_increase_drop = 0.00
        # END OF HYPER PARAMETERS
        
        self.encoder = EncoderQTPi(self.increase_drop_prob, self.stochastic_depth, self.attention_type, use_ASPP = self.use_ASPP, encoder_type = self.model_type, act = self.act)
        self.decoder = DecoderQTPi(self.num_classes, self.attention_type, self.act, use_pixel_shuffle = self.use_pixel_shuffle, stochastic_depth = self.decoder_stoc, drop_prob = self.decoder_drop, increase_drop = self.decoder_increase_drop, increase_stochastic = self.decoder_increase_stoc)
        
    def forward(self, x):
        return torch.squeeze(self.decoder(*self.encoder(x)))
        

In [ ]:
class TrainingConfig:
    model_name = 'QTPi' # QTPi = Powerful, BaseLine is Simple Transfer Learned
    optim = 'adam'
    criterion_type = 'lovask'
    lr = 1e-3
    weight_decay = 1e-3
    
    num_steps = 5
    step_size = 0.9
    eta_min = 1e-7

In [ ]:
class TrainingSolverQTPi(pl.LightningModule):
    def __init__(self, dev):
        super().__init__()
        self.dev = dev
        self.config = TrainingConfig
        self.model_name = self.config.model_name
        assert self.model_name in ['baseline', 'QTPi']
        self.model = self.configure_model()
        # Send Model to Device
        self.to(self.dev)
    def forward(self, x):
        self.eval()
        with torch.no_grad():
            pred = torch.sigmoid(self.model(x))
            return pred 
    def configure_model(self):
        '''
        Loads in the Model
        '''
        if self.model_name == 'baseline':
            model = BaseLineSolution()
        else:
            model = UNetQTPi()
        return model
       

In [ ]:
class EnsembleModel(pl.LightningModule):
    def __init__(self, model_paths, dev):
        super().__init__()
        self.dev = dev
        self.out_size = 1024
        self.model_paths = model_paths
        self.divide = len(self.model_paths)
        self.models = [TrainingSolverQTPi(self.dev) for i in range(self.divide)]
        for model_idx in range(len(self.models)):
            self.models[model_idx].load_state_dict(torch.load(self.model_paths[model_idx], map_location = self.dev))
        self.to(self.dev)
    def tta(self, x):
        im_flip1 = x.flip(-1)
        im_flip2 = x.flip(-2)
        im_flip3 = x.flip(-2).flip(-1)
        return im_flip1, im_flip2, im_flip3
    def forward_dl(self, dl):
        pred_kidney = np.zeros((len(dl.dataset), self.out_size, self.out_size), dtype = np.uint8)
        count_idx = 0
        for images, idx in dl:
            B, _, _, _ = images.shape
            bool_0 = idx == -1
            images = images.to(self.dev)
            with torch.cuda.amp.autocast():
                predictions = self(images)
            predictions[bool_0, :, :] = 0 # Remove impossible images
            pred_kidney[count_idx: count_idx + B, :, :] = predictions.detach().cpu().numpy().astype(np.uint8)
            del images
            del predictions
            del idx
            gc.collect()
            count_idx += B
            print(f"Predicted: {count_idx}")
        return pred_kidney
            
            
            
    def forward(self, x):
        self.eval()
        with torch.no_grad():
            predictions = None
            for model in self.models:
                if predictions is None:
                    predictions = self.forward_one_model(model, x)
                else:
                    predictions = predictions + self.forward_one_model(model, x)
            predictions = predictions / self.divide
            
            # Interpolate upwards
            predictions = torch.squeeze(F.interpolate(predictions.unsqueeze(1), scale_factor = reduce))
            # Threshold
            ones_bools = predictions >= 0.5
            predictions[:, :, :] = 0
            predictions[ones_bools] = 1
            return predictions
    def forward_one_model(self, model, x):
        return model(x)

# Prediction

In [ ]:
model = EnsembleModel(MODELS, device)

In [ ]:
names,preds = [],[]
for idx,row in tqdm(df_sample.iterrows(),total=len(df_sample)):
    idx = row['id']
    ds = HuBMAPDataset(idx)
    #rasterio cannot be used with multiple workers
    dl = DataLoader(ds,bs,num_workers=0,shuffle=False,pin_memory=True)
    # Ignore Images with Index -1(White or Black only)
    #generate masks
    # Predict, then resize upwards using BiLinear Interpolation to 1024x1024.
    # Mask should be of size(DatasetLength, 1024, 1024)
    #reshape tiled masks into a single mask and crop padding
    mask = model.forward_dl(dl)
    
    mask = mask.reshape(ds.n0max,ds.n1max,ds.sz,ds.sz).\
        transpose(0,2,1,3).reshape(ds.n0max*ds.sz,ds.n1max*ds.sz)
    mask = mask[ds.pad0//2:-(ds.pad0-ds.pad0//2) if ds.pad0 > 0 else ds.n0max*ds.sz,
        ds.pad1//2:-(ds.pad1-ds.pad1//2) if ds.pad1 > 0 else ds.n1max*ds.sz]
    
    plt.imshow(mask)
    plt.show()
    #convert to rle
    #https://www.kaggle.com/bguberfain/memory-aware-rle-encoding
    rle = rle_encode_less_memory(mask)
    names.append(idx)
    preds.append(rle)
    del mask, ds, dl
    gc.collect()


In [ ]:
df = pd.DataFrame({'id':names,'predicted':preds})
df.to_csv('submission.csv',index=False)